In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','<','br','>','Q','A']) 

In [2]:
import os, sys
import warnings
warnings.filterwarnings('ignore')

In [3]:
from xml.sax.saxutils import unescape
import xml.etree.ElementTree as ET
import readability
import html.parser
doc = open('CutData/Pseudo/NTF_200/Features_NTF2_1/Doc_NTF2_1.csv', 'r').read() 
doc_num = doc.split('\n')  
tree = ET.parse('Formspring/new_XMLMergedFile.xml')
root = tree.getroot()

In [4]:
import csv
def store(data,data2,data3,data4,data5,data6):
    with open('CutData/Pseudo/NTF_200/Features_NTF2_1/Readability_NTF2_1.csv', 'a', newline='') as csvfile:
        fieldnames = ['FRE','SMOG','CLI','ARI','FOG','FKGL']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for line,line2,line3,line4,line5,line6 in zip(data,data2,data3,data4,data5,data6):
            writer.writerow({'FRE': str(line),'SMOG': str(line2),'CLI': str(line3) 
                            ,'ARI': str(line4),'FOG': str(line5) ,'FKGL': str(line6)})

In [5]:
fre_list=[]
smog_list=[]
cli_list=[]
ari_list=[]
fog_list=[]
fkgl_list=[]
for child in root.iter(tag='TEXT'):
    if child.attrib["id"] in doc_num:
        html_parser = html.parser.HTMLParser()
        content = html_parser.unescape(child.text)
        results = readability.getmeasures(content, lang='en')
        fre_list.append(results['readability grades']['FleschReadingEase'])
        smog_list.append(results['readability grades']['SMOGIndex'])
        cli_list.append(results['readability grades']['Coleman-Liau'])
        ari_list.append(results['readability grades']['ARI'])
        fog_list.append(results['readability grades']['GunningFogIndex'])
        fkgl_list.append(results['readability grades']['Kincaid'])
store(fre_list,smog_list,cli_list,ari_list,fog_list,fkgl_list)    
#     print("FRE:",results['readability grades']['FleschReadingEase'])
#     print("SMOG",results['readability grades']['SMOGIndex'])
#     print("CLI",results['readability grades']['Coleman-Liau'])
#     print("ARI",results['readability grades']['ARI'])
#     print("FOG",results['readability grades']['GunningFogIndex'])
#     print("FKGL",results['readability grades']['Kincaid'])

In [6]:
import csv
def word_store(data):
    with open('CutData/Pseudo/NTF_200/Features_NTF2_1/Length_NTF2_1.csv', 'a', newline='') as csvfile:
        fieldnames = ['Length']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for line in data:
            writer.writerow({'Length': str(line)})

In [7]:
word_list=[]
for child in root.iter(tag='TEXT'):
    if child.attrib["id"] in doc_num:
        html_parser = html.parser.HTMLParser()
        content = html_parser.unescape(child.text)
        results = readability.getmeasures(content, lang='en')
        word_list.append(results['sentence info']['words'])
word_store(word_list)

In [8]:
import csv
def spell_store(data):
    with open('CutData/Pseudo/NTF_200/Features_NTF2_1/SpellMiss_NTF2_1.csv', 'a', newline='') as csvfile:
        fieldnames = ['SpellMiss(num)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for line in data:
            writer.writerow({'SpellMiss(num)': str(line)})

In [9]:
from spellchecker import SpellChecker
import re
word_list=[]
symbol = {'!', '"', '#', '$', '%', '&',"'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', 
          '=', '>', '?', '@', '[',  ']', '^', '_', '`', '{', '|', '}', '~', '\'' ,'<br>' ,'Q','A'}
spell = SpellChecker()
for child in root.iter(tag='TEXT'):
    if child.attrib["id"] in doc_num:
        html_parser = html.parser.HTMLParser()
        content = html_parser.unescape(child.text)
        re_url = re.sub(r'http://[a-zA-Z0-9./?%&_=-]*','' ,content)
        for x in symbol:
            if x in re_url:
                re_url = re_url.replace(x, '').strip()    #去除符號
        stokens = nltk.word_tokenize(re_url)#斷詞
        misspelled = spell.unknown(stokens)
        word_list.append(len(misspelled))
spell_store(word_list)   